# BORROWER'S CREDIT RISK MODEL

In this project, my aim is creating a borrower's credit risk model for lenders. The Lending Club releases all data which contains approved or declined loan aplications. You can download the data setes from this [website](https://www.lendingclub.com/auth/login?login_url=%2Fstatistics%2Fadditional-statistics%3F). Let's start with exploring and cleaning data.

### Opening and Clearing Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [2]:
loans_2007= pd.read_csv("loans_2007.csv")
loans_2007.drop_duplicates()
loans_2007.iloc[0]


C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

In [3]:
print(loans_2007.shape[1])

52


There are a lot of columns in the data set and some of them is seen useless. I have to decide which column is useful for our prediction model. 

In [4]:
# We will drop some colmns which is in cols list.
cols= ["id",
"member_id",
"funded_amnt",
"funded_amnt_inv",
"grade",
"sub_grade",
"emp_title",
"issue_d",
"zip_code",
"out_prncp",
"out_prncp_inv",
"total_pymnt",
"total_pymnt_inv",
"total_rec_prncp",
"total_rec_int",
"total_rec_late_fee",
"recoveries",
"collection_recovery_fee",
"last_pymnt_d",
"last_pymnt_amnt"]

In [5]:
loans_2007= loans_2007.drop(cols,axis=1)

In [6]:
loans_2007.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42538 entries, 0 to 42537
Data columns (total 32 columns):
loan_amnt                     42535 non-null float64
term                          42535 non-null object
int_rate                      42535 non-null object
installment                   42535 non-null float64
emp_length                    41423 non-null object
home_ownership                42535 non-null object
annual_inc                    42531 non-null float64
verification_status           42535 non-null object
loan_status                   42535 non-null object
pymnt_plan                    42535 non-null object
purpose                       42535 non-null object
title                         42522 non-null object
addr_state                    42535 non-null object
dti                           42535 non-null float64
delinq_2yrs                   42506 non-null float64
earliest_cr_line              42506 non-null object
inq_last_6mths                42506 non-null float64
o

Now, I have 32 columns to use. The 'loan_status' column shows that loan is accepted or dismissed. But this column has text type values. Let's change the column type from text to numerical. 

In [7]:
loans_2007["loan_status"].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

I will use only columns which has 'Charged Off' and 'Fully Paid' values. Because, my aim is to make prediction about loan which is paid or is not paid. Other values do not give proper information for our model. I will start with dropping these entries which have values out of our need.

In [8]:
loans_2007= loans_2007[(loans_2007["loan_status"]== "Fully Paid")|(loans_2007["loan_status"]== "Charged Off")]

In [9]:
loans_2007.shape

(38770, 32)

I will use binary system.

In [10]:
status_replace= {"loan_status": {"Fully Paid": 1,"Charged Off": 0,}}
loans_2007= loans_2007.replace(status_replace)


In [11]:
loans_2007.loan_status.value_counts()

1    33136
0     5634
Name: loan_status, dtype: int64

After that, I will drop the columns which have only one unique value. These columns are not able to give information for the model.

In [12]:
orig_columns= loans_2007.columns
drop_columns=[]
for col in orig_columns:
    col_series= loans_2007[col].dropna().unique()
    if len(col_series)== 1:
        drop_columns.append(col)
        
loans_2007= loans_2007.drop(drop_columns, axis=1)
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


### Preparing Features 

In [13]:
loans= loans_2007
loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38770 entries, 0 to 39785
Data columns (total 23 columns):
loan_amnt               38770 non-null float64
term                    38770 non-null object
int_rate                38770 non-null object
installment             38770 non-null float64
emp_length              37734 non-null object
home_ownership          38770 non-null object
annual_inc              38770 non-null float64
verification_status     38770 non-null object
loan_status             38770 non-null int64
purpose                 38770 non-null object
title                   38759 non-null object
addr_state              38770 non-null object
dti                     38770 non-null float64
delinq_2yrs             38770 non-null float64
earliest_cr_line        38770 non-null object
inq_last_6mths          38770 non-null float64
open_acc                38770 non-null float64
pub_rec                 38770 non-null float64
revol_bal               38770 non-null float64
revol_uti

Some columns have missing values, we have to get rid of these missing values. I will detect the missing values and develop a strategy to clean them.

In [14]:
loans.sample(2)

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,...,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_credit_pull_d,pub_rec_bankruptcies
33402,25000.0,36 months,14.96%,866.13,2 years,MORTGAGE,87000.0,Not Verified,1,debt_consolidation,...,0.0,Dec-1980,3.0,17.0,0.0,13580.0,48.5%,49.0,Dec-2012,0.0
19522,6000.0,36 months,14.54%,206.65,5 years,RENT,40000.0,Not Verified,1,debt_consolidation,...,0.0,Jul-1986,3.0,26.0,0.0,2045.0,4.8%,47.0,May-2015,0.0


In [15]:
null_counts= loans.isnull().sum()
print(null_counts[null_counts>0])

emp_length              1036
title                     11
revol_util                50
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


In [16]:
import seaborn as sns
import matplotlib.pyplot as plt
ax = sns.countplot(x="loan_status", data=loans[loans['emp_length'].isnull()])
plt.show()

<Figure size 640x480 with 1 Axes>

In [17]:
loans[loans['emp_length'].isnull()]['loan_status'].sum()

808

In [18]:
#The "pub_rec_bankruptcies" column will be dropped.
#Other rows which have missing values will be dropped.
loans= loans.drop("pub_rec_bankruptcies", axis=1)
loans= loans.dropna(axis=0)


In [19]:
loans.loan_status.value_counts()

1    32286
0     5389
Name: loan_status, dtype: int64

In [20]:
print(loans.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


I need only numerical data types to use. I will explore the columns which are object data type. 

In [21]:
object_columns_df= loans.select_dtypes(include=["object"])
print(object_columns_df.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Jun-2016
Name: 0, dtype: object


In [22]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for c in cols:
    print('Column', c)
    print(loans[c].value_counts())
    print('=================')

Column home_ownership
RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Column verification_status
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
Column emp_length
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
Column term
 36 months    28234
 60 months     9441
Name: term, dtype: int64
Column addr_state
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
AL     420
LA     420
KY     311
OK     285
UT     249
KS     249
AR     229
DC     209
RI     194
NM    

In [23]:
print(loans["title"].value_counts())
print(loans["purpose"].value_counts())

Debt Consolidation                          2068
Debt Consolidation Loan                     1599
Personal Loan                                624
Consolidation                                488
debt consolidation                           466
Credit Card Consolidation                    345
Home Improvement                             336
Debt consolidation                           314
Small Business Loan                          298
Credit Card Loan                             294
Personal                                     290
Consolidation Loan                           250
Home Improvement Loan                        228
personal loan                                219
Loan                                         202
Wedding Loan                                 199
personal                                     198
Car Loan                                     188
consolidation                                186
Other Loan                                   168
Wedding             

In [24]:
#We will use mapping method to encode the 'emp_length' column.
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

loans= loans.replace(mapping_dict)

In [25]:
#We will drop some columns again.
loans= loans.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)

In [26]:
#We will clean "%" character from two columns.
loans["int_rate"]= loans["int_rate"].str.rstrip("%").astype("float")
loans["revol_util"]= loans["revol_util"].str.rstrip("%").astype("float")

In [27]:
#We will encode the columns which are in the list below.
cat_columns=["home_ownership", "verification_status", "purpose", "term"]
dummy_df= pd.get_dummies(loans[cat_columns])
loans= pd.concat([loans, dummy_df], axis=1)
loans= loans.drop(cat_columns, axis=1)

In [28]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37675 entries, 0 to 39785
Data columns (total 38 columns):
loan_amnt                              37675 non-null float64
int_rate                               37675 non-null float64
installment                            37675 non-null float64
emp_length                             37675 non-null object
annual_inc                             37675 non-null float64
loan_status                            37675 non-null int64
dti                                    37675 non-null float64
delinq_2yrs                            37675 non-null float64
inq_last_6mths                         37675 non-null float64
open_acc                               37675 non-null float64
pub_rec                                37675 non-null float64
revol_bal                              37675 non-null float64
revol_util                             37675 non-null float64
total_acc                              37675 non-null float64
home_ownership_MORTGAGE   

Now, I have 38 columns and 37675 entries to use.

### Prediction Step

I will use FPR and TPR metrics to observe our model's success. I want to get money. I do not give money to wrong person. If I give money to wrong person we will lose money. So, my TPR value must be high and my FPR value must be low.

In [29]:
#We will start with making prediction.
predictions = pd.Series(np.ones(loans.shape[0]))

In [30]:
#We will calculate TPR and FPR values.
fp_filter= (predictions==1) & (loans["loan_status"]==0)
fp= len(predictions[fp_filter])
tp_filter= (predictions==1) & (loans["loan_status"]==1)
tp= len(predictions[tp_filter])
fn_filter=(predictions==0) & (loans["loan_status"]==1)
fn= len(predictions[fn_filter])
tn_filter=(predictions==0) & (loans["loan_status"]==0)
tn= len(predictions[tn_filter])
tpr= tp/ (tp+fn)
fpr= fp/ (fp+ tn)

print(tpr)
print(fpr)

1.0
1.0


In [31]:
confusion_matrix(predictions, loans["loan_status"])

array([[    0,     0],
       [ 5389, 32286]], dtype=int64)

In [32]:
predictions.value_counts()

1.0    37675
dtype: int64

I have found two of FPR and TPR values as 1. Because I predicted all of the values as 1. It shows that my prediction is correctly for good results, but it is not true for bad results. After this step, I will use machine learning algorithm to improve our model. I will start first with logistic regression.

In [33]:
lr= LogisticRegression()

In [34]:
cols= loans.columns
train_cols= cols.drop("loan_status")
features= loans[train_cols]
target= loans["loan_status"]
lr.fit(features, target)
predictions= lr.predict(features)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [35]:
fp_filter= (predictions==1) & (loans["loan_status"]==0)
fp= len(predictions[fp_filter])
tp_filter= (predictions==1) & (loans["loan_status"]==1)
tp= len(predictions[tp_filter])
fn_filter= (predictions== 0)& (loans["loan_status"]==1)
fn= len(predictions[fn_filter])
tn_filter= (predictions==0)&(loans["loan_status"]==0)
tn= len(predictions[tn_filter])
tpr= tp/ (tp+fn)
fpr= fp/ (fp+tn)
print(tpr)
print(fpr)

0.9986062070247166
0.9964742994989794


I will make cross validation prediction to improve it.


In [36]:
confusion_matrix(predictions, loans["loan_status"])

array([[   19,    45],
       [ 5370, 32241]], dtype=int64)

In [37]:
predictions= cross_val_predict(lr, features, target, cv=3)
predictions= pd.Series(predictions)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [38]:
fp_filter= (predictions==1) & (loans["loan_status"]==0)
fp= len(predictions[fp_filter])
tp_filter= (predictions==1) & (loans["loan_status"]==1)
tp= len(predictions[tp_filter])
fn_filter= (predictions== 0)& (loans["loan_status"]==1)
fn= len(predictions[fn_filter])
tn_filter= (predictions==0)&(loans["loan_status"]==0)
tn= len(predictions[tn_filter])
tpr= tp/ (tp+fn)
fpr= fp/ (fp+tn)
print(tpr)
print(fpr)

0.9984268484530676
0.9986179664363277


In [39]:
confusion_matrix(predictions, loans["loan_status"])

array([[   20,    39],
       [ 5369, 32247]], dtype=int64)

In [40]:
print(predictions.head(10), loans["loan_status"].head(10))

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int64 0     1
1     0
2     1
3     1
5     1
6     1
7     1
8     0
9     0
10    1
Name: loan_status, dtype: int64


In [41]:
predictions.value_counts()

1    37616
0       59
dtype: int64

While you can see my model predicted the almost every values as  1. Let's perform cross validation process by setting class_weight parameter as balanced. 

In [42]:
lr =LogisticRegression(class_weight="balanced")
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [43]:
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])


tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])


fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])


tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])


tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.6272286313581542
0.6152023692003948


In [44]:
confusion_matrix(predictions, loans["loan_status"])

array([[ 3326, 10857],
       [ 2063, 21429]], dtype=int64)

I improved FPR value. And my TPR value decreased, also. My FPR value 61%, it is improved. Now, I can detect more unpaid payments. As you can see, I predicted 3326 true unpaid payment, but I lost 2063 unpaid payment. It means that my damage is too much. I have to drop our damage. So, I do not want this result. I will change the balance of class_weight.

In [45]:
penalty = {
    0: 12,
    1: 1}
lr = LogisticRegression(class_weight=penalty, C=0.01)
predictions = cross_val_predict(lr, features, target, cv=15)
predictions = pd.Series(predictions)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

In [46]:
fp_filter= (predictions==1) & (loans["loan_status"]==0)
fp= len(predictions[fp_filter])
tp_filter= (predictions==1) & (loans["loan_status"]==1)
tp= len(predictions[tp_filter])
fn_filter= (predictions== 0)& (loans["loan_status"]==1)
fn= len(predictions[fn_filter])
tn_filter= (predictions==0)&(loans["loan_status"]==0)
tn= len(predictions[tn_filter])
tpr= tp/ (tp+fn)
fpr= fp/ (fp+tn)
print(tpr)
print(fpr)

0.11038280020975354
0.11688055281342546


In [47]:
confusion_matrix(predictions, loans["loan_status"])

array([[ 5197, 28263],
       [  192,  4023]], dtype=int64)

I have dropped the FPR value, but my TPR is too small, also. We predicted 5197 unpaid payments truly, and we couldn't predict only 192 unpaid payment. So, we can say that this model is more succesful. We will try another machine learning algorithm. With logistic regression we can only work with linear data. However we can work on nonlinear data with random forests. 

In [48]:
rf=RandomForestClassifier(class_weight= "balanced", random_state=101)
predictions = cross_val_predict(rf, features, target, cv=10)
predictions = pd.Series(predictions)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

In [49]:
fp_filter= (predictions==1) & (loans["loan_status"]==0)
fp= len(predictions[fp_filter])
tp_filter= (predictions==1) & (loans["loan_status"]==1)
tp= len(predictions[tp_filter])
fn_filter= (predictions== 0)& (loans["loan_status"]==1)
fn= len(predictions[fn_filter])
tn_filter= (predictions==0)&(loans["loan_status"]==0)
tn= len(predictions[tn_filter])
tpr= tp/ (tp+fn)
fpr= fp/ (fp+tn)
print(tpr)
print(fpr)

0.9659805977975878
0.9642645607107602


In [50]:
print(confusion_matrix(predictions, loans["loan_status"]))

[[  384   916]
 [ 5005 31370]]


### Summary

In this project my aim is to reduce the FPR value. After using different models and parametrics, my logistic regression model is more successful. I set class_weight parameter as penalty and set C parameter as 0.01. And I found the FPR value as 11%. It is the lowest rate I found. I predicted 5197 unpaid payment truly. 